In [60]:
import numpy as np
import pandas as pd
import streamlit as st
import streamlit.components.v1 as components
import pandas as pd
import networkx as nx
from pyvis.network import Network
from scipy.stats import zscore
df_data = pd.read_csv('data/transaction_data.csv')
df_data.head(10)

,chemical,amount,agency,contractor
0,Corrosion,1662974.15,DEPT OF THE AIR FORCE,A & L SANCHEZ PAINTING AND CONSTRUCTION INC
1,Corrosion,646074.99,DEPT OF THE AIR FORCE,A & L SANCHEZ PAINTING AND CONSTRUCTION INC
2,Corrosion,3900000,DEPT OF THE AIR FORCE,A & L SANCHEZ PAINTING AND CONSTRUCTION INC
3,Coating,1290,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
4,Coating,129,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
5,Coating,384.64,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
6,Corrosion,1014,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
7,Coating,63.76,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
8,Corrosion,1014,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC
9,Coating,129,DEFENSE LOGISTICS AGENCY,A.M.S. NETWORK LLC


In [32]:
df_data['amount'] = pd.to_numeric(df_data['amount'], errors='coerce')
df_data['amount'].dtypes

dtype('float64')

In [40]:
x = df_data.groupby(['chemical', 'agency', 'contractor']).agg({'amount': ['sum']})
x = x.reset_index()
x

,chemical,agency,contractor,amount
,,,,sum
0,Anti-icing / Deicing product,AGRICULTURAL RESEARCH SERVICE,HOMELAND INDUSTRIAL SUPPLY INC.,0.00
1,Anti-icing / Deicing product,DEFENSE LOGISTICS AGENCY,ALCOM MEDIKAL BILGISAYAR INS TUR OTO TEKS PAZ ...,0.00
2,Anti-icing / Deicing product,DEFENSE LOGISTICS AGENCY,BOSMA INDUSTRIES FOR THE BLIND INC.,990.00
3,Anti-icing / Deicing product,DEFENSE LOGISTICS AGENCY,DC GALAXY HOLDING C.V.,0.00
4,Anti-icing / Deicing product,DEFENSE LOGISTICS AGENCY,DEFENSE ENERGY SYNDICATE LLC,0.00
...,...,...,...,...
532,Salt / Reagent,"VETERANS AFFAIRS, DEPARTMENT OF",MJL ENTERPRISES LLC,1919836.98
533,Salt / Reagent,"VETERANS AFFAIRS, DEPARTMENT OF",MONROE COUNTY OF (INC),44286.58
534,Salt / Reagent,"VETERANS AFFAIRS, DEPARTMENT OF",R & M GOVERNMENT SERVICES INC.,3638400.00


## This is where the magic is happening

In [177]:
selected_chem = 'Salt / Reagent'
x_sel = x.loc[x['chemical'] == selected_chem]
x_sel = x_sel.reset_index()

g = Network()
f = x_sel['agency'].unique().tolist()
agency_length = len(f)
m = x_sel['contractor'].unique().tolist()
tot_list = f + m

z = 0
for point in range(len(f)):
    g.add_node(z, label=f[point], color='#F17B7B')
    z += 1
for points in range(len(m)):
    g.add_node(z, label=m[points], color='#6F7CDB')
    z += 1


df_track = pd.DataFrame(tot_list)
agen_list = x_sel['agency'].tolist()
cont_list = x_sel['contractor'].tolist()

d = x_sel['amount'].apply(zscore).values.tolist()
weight_list = []
for numbers in range(len(d)):
    weight_list.append(d[numbers][0])
floor = min(weight_list)
wt_list = [x-floor for x in weight_list]

for t in range(len(agen_list)):
    agen_list[t] = df_track.index[df_track[0] == agen_list[t]].tolist()[0]
    
for u in range(len(cont_list)):
    cont_list[u] = df_track.index[df_track[0] == cont_list[u]].tolist()[0]

for i in range(len(agen_list)):
    g.add_edge(agen_list[i], cont_list[i], weight=wt_list[i])

## And this is all scratch pad

In [176]:
from scipy.stats import zscore
d = x_sel['amount'].apply(zscore).values.tolist()
weight_list = []
for numbers in range(len(d)):
    weight_list.append(d[numbers][0])
floor = min(weight_list)
wt_list = [x-floor for x in weight_list]
wt_list

[0.010344189075526145,
 0.05514641983039603,
 1.808960363114398e-05,
 0.02462005148039781,
 0.02158217472373747,
 0.09481082535600005,
 0.01309868415228177,
 0.0008709325826353753,
 9.498303636445993e-06,
 0.0,
 0.010068771435546076,
 0.0045555159135579615,
 0.02248311340935888,
 4.174514707856616,
 0.0025543515291433505,
 0.4430697605243726,
 0.014565542641066576,
 0.05456212491212131,
 0.007957716854756602,
 0.13697862437944236,
 0.0005618159361299724,
 0.9666582588727266,
 4.037526876452927,
 2.3860273636018894,
 0.13697862437944236,
 0.1177109253982202,
 0.0020949260661131675,
 0.03203729366117991,
 0.00010208188998483969,
 0.02205837903314878,
 0.21936599013996674,
 0.04292249094223438,
 0.024473295695626396,
 0.0018509435284973508,
 0.4627909670943082,
 0.6399159274001187,
 0.05945261644882216,
 0.5388127419298052,
 0.04837633700470845,
 0.012902418081286782,
 0.005276536223391026,
 4.527517995943149,
 0.003319591510953457,
 0.06156746342957575,
 0.01577179115345756,
 0.007561173